In [1]:
import h5py
%load_ext autoreload
%autoreload 2

In [2]:
from im2mesh import config
import torch
import torch.distributions as dist
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
np.random.seed(0)

In [4]:
# data preparation
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/sdf_train_core.h5'
batch_size = 64
num_of_workers = 12

occ_net_path = 'occ_net.pth'

# train

In [5]:
class ChairSDFDataset(Dataset):
     
    def __init__(self, h5_file):
        
        self.file_path = h5_file
        self.dataset = None
        
        with h5py.File(self.file_path, 'r') as file:
            self.dataset_len = len(file)
            self.keys = list(file.keys())
            
        self.to_tensor = transforms.ToTensor()
     
    def __len__(self):
        return self.dataset_len
 
    def __getitem__(self, idx):
        
        #start_time = time.time()
        
        if self.dataset is None:
            self.dataset = h5py.File(self.file_path, 'r')
         
        group = self.dataset[self.keys[idx]]
        
        depth_img = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        
        #print("--- depth preprocessing %s seconds ---" % (time.time() - start_time))
        
        sample_pt_np = np.array(group['sample_pt']).reshape(-1, 3)
        sample_sdf_np = np.array(group['sample_sdf']).reshape(-1, 1)
        
        # check size correctness and fix incorrect data
        if sample_pt_np.shape[0] != 2048:
            sample_pt_np = np.pad(sample_pt_np, ((0, 2048 - sample_pt_np.shape[0]), (0, 0)), 'reflect')
        if sample_sdf_np.shape[0] != 2048:
            sample_sdf_np = np.pad(sample_sdf_np, ((0, 2048 - sample_sdf_np.shape[0]), (0, 0)), 'reflect')
            
        
        sample_pt = torch.from_numpy(sample_pt_np).float()
        sample_occ = (torch.from_numpy(sample_sdf_np) < 0).float()
        
        sample = { 'depth_img': depth_img,
                   'sample_pt':sample_pt,
                   'sample_occ':sample_occ,
                  }
        
        return sample

In [6]:
train_sdf_dataset = ChairSDFDataset(data_file)
train_sdf_dataloader = DataLoader(train_sdf_dataset, batch_size=batch_size, shuffle=True, num_workers=num_of_workers)

In [5]:
from encoder.models import Discrete_encoder

In [6]:
from im2mesh.layers import CResnetBlockConv1d,CBatchNorm1d

class DecoderCBatchNorm(nn.Module):
    ''' Decoder with conditional batch normalization (CBN) class.

    Args:
        dim (int): input dimension
        z_dim (int): dimension of latent code z
        c_dim (int): dimension of latent conditioned code c
        hidden_size (int): hidden size of Decoder network
        leaky (bool): whether to use leaky ReLUs
        legacy (bool): whether to use the legacy structure
    '''

    def __init__(self, dim=3, z_dim=0, c_dim=256,
                 hidden_size=256, leaky=False, legacy = False):
        super().__init__()
        self.z_dim = z_dim
        if not z_dim == 0:
            self.fc_z = nn.Linear(z_dim, hidden_size)

        self.fc_p = nn.Conv1d(dim, hidden_size, 1)
        self.block0 = CResnetBlockConv1d(c_dim, hidden_size, legacy=legacy)
        self.block1 = CResnetBlockConv1d(c_dim, hidden_size, legacy=legacy)
        self.block2 = CResnetBlockConv1d(c_dim, hidden_size, legacy=legacy)
        self.block3 = CResnetBlockConv1d(c_dim, hidden_size, legacy=legacy)
        self.block4 = CResnetBlockConv1d(c_dim, hidden_size, legacy=legacy)

        self.bn = CBatchNorm1d(c_dim, hidden_size)
        
        self.fc_out = nn.Conv1d(hidden_size, 1, 1)

        if not leaky:
            self.actvn = F.relu
        else:
            self.actvn = lambda x: F.leaky_relu(x, 0.2)

    def forward(self, p, z, c):
        p = p.transpose(1, 2)
        batch_size, D, T = p.size()
        net = self.fc_p(p)

        if self.z_dim != 0:
            
            net_z = self.fc_z(z).unsqueeze(2)
            net = net + net_z

        net = self.block0(net, c)
        net = self.block1(net, c)
        net = self.block2(net, c)
        net = self.block3(net, c)
        net = self.block4(net, c)

        out = self.fc_out(self.actvn(self.bn(net, c)))
        out = out.squeeze(1)

        return out

In [7]:
class OccupancyNetwork(nn.Module):
    ''' Occupancy Network class.

    Args:
        decoder (nn.Module): decoder network
        encoder (nn.Module): encoder network
        encoder_latent (nn.Module): latent encoder network
        p0_z (dist): prior distribution for latent code z
        device (device): torch device
    '''

    def __init__(self, decoder, encoder=None, encoder_latent=None, p0_z=None,
                 device=None):
        super().__init__()
        if p0_z is None:
            p0_z = dist.Normal(torch.tensor([]), torch.tensor([]))

        self.decoder = decoder.to(device)

        if encoder_latent is not None:
            self.encoder_latent = encoder_latent.to(device)
        else:
            self.encoder_latent = None

        if encoder is not None:
            self.encoder = encoder.to(device)
        else:
            self.encoder = None

        self._device = device
        self.p0_z = p0_z

    def forward(self, p, inputs, sample=True, **kwargs):
        ''' Performs a forward pass through the network.

        Args:
            p (tensor): sampled points
            inputs (tensor): conditioning input
            sample (bool): whether to sample for z
        '''
        batch_size = p.size(0)
        c = self.encode_inputs(inputs)
        z = self.get_z_from_prior((batch_size,), sample=sample)
        p_r = self.decode(p, z, c, **kwargs)
        return p_r

    def compute_elbo(self, p, occ, inputs, **kwargs):
        ''' Computes the expectation lower bound.

        Args:
            p (tensor): sampled points
            occ (tensor): occupancy values for p
            inputs (tensor): conditioning input
        '''
        c = self.encode_inputs(inputs)
        q_z = self.infer_z(p, occ, c, **kwargs)
        z = q_z.rsample()
        p_r = self.decode(p, z, c, **kwargs)

        rec_error = -p_r.log_prob(occ).sum(dim=-1)
        kl = dist.kl_divergence(q_z, self.p0_z).sum(dim=-1)
        elbo = -rec_error - kl

        return elbo, rec_error, kl

    def encode_inputs(self, inputs):
        ''' Encodes the input.

        Args:
            input (tensor): the input
        '''

        if self.encoder is not None:
            c = self.encoder(inputs)[:, :, 0, 0, 0]
        else:
            # Return inputs?
            c = torch.empty(inputs.size(0), 0)

        return c

    def decode(self, p, z, c):
        ''' Returns occupancy probabilities for the sampled points.

        Args:
            p (tensor): points
            z (tensor): latent code z
            c (tensor): latent conditioned code c
        '''

        logits = self.decoder(p, z, c)
        p_r = dist.Bernoulli(logits=logits)
        return p_r

    def infer_z(self, p, occ, c, **kwargs):
        ''' Infers z.

        Args:
            p (tensor): points tensor
            occ (tensor): occupancy values for occ
            c (tensor): latent conditioned code c
        '''
        if self.encoder_latent is not None:
            mean_z, logstd_z = self.encoder_latent(p, occ, c, **kwargs)
        else:
            batch_size = p.size(0)
            mean_z = torch.empty(batch_size, 0).to(self._device)
            logstd_z = torch.empty(batch_size, 0).to(self._device)

        q_z = dist.Normal(mean_z, torch.exp(logstd_z))
        return q_z

    def get_z_from_prior(self, size=torch.Size([]), sample=True):
        ''' Returns z from prior distribution.

        Args:
            size (Size): size of z
            sample (bool): whether to sample
        '''
        if sample:
            z = self.p0_z.sample(size).to(self._device)
        else:
            z = self.p0_z.mean.to(self._device)
            z = z.expand(*size, *z.size())

        return z

    def to(self, device):
        ''' Puts the model to the device.

        Args:
            device (device): pytorch device
        '''
        model = super().to(device)
        model._device = device
        return model

In [8]:
encoder = Discrete_encoder()
encoder.load_state_dict(torch.load('encoder/discrete_encoder.pth'))
encoder.eval()

for child in encoder.children():
    for param in child.parameters():
        param.requires_grad = False

# b x m x 3
decoder = DecoderCBatchNorm()

dim = 3
z_dim = 0
c_dim = 256
encoder_latent = None
p0_z = dist.Normal(
        torch.zeros(z_dim, device=device),
        torch.ones(z_dim, device=device)
    )

model = OccupancyNetwork(
        decoder, encoder, encoder_latent, p0_z, device=device
    )

In [9]:
def compute_loss(p, occ, inputs):

    c = model.encode_inputs(inputs)
    q_z = model.infer_z(p, occ, c)
    z = q_z.rsample()

    # KL-divergence
    kl = dist.kl_divergence(q_z, model.p0_z).sum(dim=-1)
    loss = kl.mean()

    # General points
    logits = model.decode(p, z, c).logits
    loss_i = F.binary_cross_entropy_with_logits(logits, occ, reduction='none')
    loss = loss + loss_i.sum(-1).mean()
    
    return loss

In [12]:
start_epoch = 0
num_epoch = 40

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

In [13]:
def save_checkpoint(state):
    torch.save(state, 'continuous_model_checkpoint.pth.tar')

In [ ]:
import time


batch_len = len(train_sdf_dataloader)

print("Starting Training Loop...")

start_time = time.time()

for epoch in range(start_epoch, num_epoch):
    
    loss_list = []
    loss_batch = []
    
    print("Epoch: ", epoch)
    
    count = 0
    
    for i, data in enumerate(train_sdf_dataloader):
        
        ####################
        # Data preparation #
        ####################
        
        # b x 1 x 256 x 256
        depth_img = data['depth_img'].to(device)
        # b x n x 3
        sample_pt = data['sample_pt'].to(device)
        # b x n
        sample_occ = data['sample_occ'].squeeze(-1).to(device)
        
        optimizer.zero_grad()
        
        loss = compute_loss(sample_pt, sample_occ, depth_img)
        
        loss.backward()
        
        loss_list.append(loss.item())
        loss_batch.append(loss.item())
        
        optimizer.step()
        
        #scheduler.step()
        
        if count != 0 and count % 100 == 0:
            loss_batch_avg = np.average(loss_batch)
            
            print("Batch: ", count, ", BCE Loss: ", loss_batch_avg, ", Time: %s s" % (time.time() - start_time))
            
            if count % 500 == 0:
                torch.save(model.state_dict(), occ_net_path)
                
            loss_batch.clear()
            
        count += 1
        
    print("Epoch: ", epoch, ', BCE loss: ', np.average(loss_list))
    
    loss_list.clear()
    
    torch.save(model.state_dict(), occ_net_path)
    
    save_checkpoint({
        'epoch': epoch + 1,
        'occ_net_path_state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    })
    
print("Training finished")

Starting Training Loop...
Epoch:  0
Batch:  100 , BCE Loss:  1184.1777972230816 , Time: 30.994378089904785 s
Batch:  200 , BCE Loss:  1083.9680737304689 , Time: 61.46299171447754 s
Batch:  300 , BCE Loss:  1064.7080615234374 , Time: 92.19846749305725 s
Batch:  400 , BCE Loss:  1056.9759765625 , Time: 123.16960525512695 s
Batch:  500 , BCE Loss:  1049.928231201172 , Time: 154.29688429832458 s
Batch:  600 , BCE Loss:  1044.2420837402344 , Time: 185.60796785354614 s
Batch:  700 , BCE Loss:  1039.6613598632812 , Time: 217.0192801952362 s
Batch:  800 , BCE Loss:  1038.095453491211 , Time: 248.64897966384888 s
Batch:  900 , BCE Loss:  1034.632496948242 , Time: 280.35790634155273 s
Batch:  1000 , BCE Loss:  1032.8912103271484 , Time: 312.1955027580261 s
Batch:  1100 , BCE Loss:  1028.78201171875 , Time: 344.16581678390503 s
Batch:  1200 , BCE Loss:  1028.81498046875 , Time: 376.24730944633484 s
Batch:  1300 , BCE Loss:  1025.3064868164063 , Time: 408.3184838294983 s
Epoch:  0 , BCE loss:  105

Batch:  500 , BCE Loss:  962.9866784667969 , Time: 3619.886739253998 s
Batch:  600 , BCE Loss:  963.187583618164 , Time: 3655.1926946640015 s
Batch:  700 , BCE Loss:  962.8619421386719 , Time: 3691.36852312088 s
Batch:  800 , BCE Loss:  962.2321789550781 , Time: 3725.5339477062225 s
Batch:  900 , BCE Loss:  963.964398803711 , Time: 3758.665347099304 s
Batch:  1000 , BCE Loss:  961.5827459716797 , Time: 3792.3923773765564 s
Batch:  1100 , BCE Loss:  960.3809747314453 , Time: 3824.742283344269 s
Batch:  1200 , BCE Loss:  962.0572589111329 , Time: 3857.0522768497467 s
Batch:  1300 , BCE Loss:  962.8651519775391 , Time: 3889.2011609077454 s
Epoch:  8 , BCE loss:  963.1493688770198
Epoch:  9
Batch:  100 , BCE Loss:  960.2855327341816 , Time: 3927.6040892601013 s
Batch:  200 , BCE Loss:  961.7588110351562 , Time: 3962.290081501007 s
Batch:  300 , BCE Loss:  961.7478546142578 , Time: 3997.0817477703094 s
Batch:  400 , BCE Loss:  960.1990112304687 , Time: 4032.9624030590057 s
Batch:  500 , BCE

In [10]:
model.load_state_dict(torch.load(occ_net_path))
model.eval()

OccupancyNetwork(
  (decoder): DecoderCBatchNorm(
    (fc_p): Conv1d(3, 256, kernel_size=(1,), stride=(1,))
    (block0): CResnetBlockConv1d(
      (bn_0): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (bn_1): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (fc_0): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (fc_1): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (actvn): ReLU()
    )
    (block1): CResnetBlockConv1d(
      (bn_0): CBatchNorm1d(
        (conv_gamma): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
        (conv_beta): Conv1d(256,

In [11]:
class ChairDepthDataset(Dataset):
    
    def __init__(self, h5_file):
        
        self.hf = h5py.File(h5_file, 'r')
        self.keys = list(self.hf.keys())
        
        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        
        group = self.hf[self.keys[idx]]
        
        model_id = str(self.keys[idx])
        depth_img = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        azimuth = float(group['azimuth'][()])
        elevation = float(group['elevation'][()])
        distance = float(group['distance'][()])
        target_vox = torch.tensor(group['target_vox'], dtype=torch.float)
        
        sample = {'model_id': model_id,
                  'depth_img': depth_img,
                  'azimuth': azimuth,
                  'elevation': elevation,
                  'distance': distance,
                  'target_vox': target_vox, 
                 }

        return sample

In [12]:
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/data_test_rescale.h5'

test_depth_dataset = ChairDepthDataset(data_file)

In [93]:
# plot vox or img
from vis_utils import plotFromVoxels, plotImg, plot_image_list
# generate with continuous model
from render_utils import generate_mesh, get_relative_transform_matrix
# render mesh with pyrender
from render_utils import render
# mesh from ground truth vox
from render_utils import RotateAlongAxis
# transfrom sdf
from render_utils import get_transformed_indices, sdf2Voxel, get_meshgrid, get_transformed_meshgrid, get_relative_transformed_vox
# get cd, emd, iou from 2 pymesh
from render_utils import get_test_results

from refine_utils import get_radius, get_depth_close_idx

In [84]:
import time
import pymesh
import trimesh
import skimage.measure as sk

In [96]:
sample_size = 2048

# generate mesh from continuous model
def generate_mesh(img, model, device, vox_res = 32, grid_res = 64, batch_size = 32, azimuth = 0, elevation = 0, isosurface = 0.5):

    vox = np.zeros((grid_res, grid_res, grid_res))
    idx = np.array(np.where(vox == 0))
    
    # normalize
    sample_pt = (torch.t(torch.tensor(idx/grid_res, dtype=torch.float)) - 0.5)
    sample_pt = sample_pt.reshape(-1, sample_size, 3)
    
    sample_pt_normalized = sample_pt + torch.tensor([0.5, 0.5, 0.5])
    
    pre_occ_list = []
    
    for i in range(int(sample_pt.shape[0]/batch_size)):
        
        start = i*batch_size
        end = (i + 1)*batch_size
        
        sample_pt_batch = sample_pt[start:end, :, :].to(device)
        img_batch = img.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)
        
        pred_occ_batch = model(sample_pt_batch, img_batch).probs.squeeze(1).detach().cpu()
        
        pre_occ_list.append(pred_occ_batch)
        
    pred_occ = torch.cat(pre_occ_list).reshape(-1,)
    
    vox[tuple([idx[0], idx[1], idx[2]])] = pred_occ[:].numpy()
    

    try:
        verts, faces, _, _ =  sk.marching_cubes_lewiner(vox, level=0.5)
        mesh = trimesh.Trimesh(verts, faces)
        trimesh.repair.fill_holes(mesh)
        trimesh.repair.fix_inversion(mesh)
        mesh_py = pymesh.form_mesh(mesh.vertices, mesh.faces)
        return mesh_py
    except:
        print("Failed generation")
        return None

In [60]:
def evaluate(idx):
    
    data = test_depth_dataset[24*idx]
    depth_img = data['depth_img']
    target_vox = data['target_vox']
    
    render()

In [97]:
def evaluate(idx):

    data_from = test_depth_dataset[24*idx]
    depth_img_from = data_from['depth_img']
    distance_from = data_from['distance']
    azimuth_from = data_from['azimuth']
    elevation_from = data_from['elevation']
    
    radius = get_radius(depth_img_from, distance_from)
    depth_index_from, close_index_from = get_depth_close_idx(depth_img_from, distance_from, radius)

    model_dir = path_model + test_depth_dataset[24*idx]['model_id'].split('_')[0] + '/model.obj'
    mesh_py = pymesh.load_mesh(model_dir)
    transformed_vertices = get_transformed_indices(mesh_py.vertices, azimuth_from, elevation_from, 1)
    gt_radius = np.max(np.linalg.norm(transformed_vertices, axis = 1))
    mesh_gt = pymesh.form_mesh(transformed_vertices/gt_radius, mesh_py.faces)
    
    return mesh_gt, generate_mesh(depth_img_from, model, device)

In [104]:
def normalize_mesh(mesh_py, vox_size = 64):
    
    normalized_vertices = mesh_py.vertices - (vox_size/2)
    normalized_vertices /= np.max(np.linalg.norm(normalized_vertices, axis = 1))
    
    return pymesh.form_mesh(normalized_vertices, mesh_py.faces)

In [99]:
path_model = '/home/ankbzpx/datasets/ShapeNet/ShapeNetCore.v1/03001627/'



In [105]:
cd_gt_v_b_list = []
emd_gt_v_b_list = []
iou_gt_v_b_list = []

failed_idx = []

for idx in range(int(len(test_depth_dataset)/24)):
    print(idx)
    start_time = time.time()
    
    mesh_gt, mesh = evaluate(idx)
    
    
    if mesh_gt is None or mesh is None:
        print('Failed case')
        failed_idx.append(idx)
        continue
        
    mesh = normalize_mesh(mesh, 64)
    
    cd_gt_v_b, emd_gt_v_b, iou_gt_v_b = get_test_results(mesh_gt, mesh)
    
    cd_gt_v_b_list.append(cd_gt_v_b)
    emd_gt_v_b_list.append(emd_gt_v_b)
    iou_gt_v_b_list.append(iou_gt_v_b)
    
    print("--- %s seconds ---" % (time.time() - start_time))

0
--- 0.9991950988769531 seconds ---
1
--- 0.5479283332824707 seconds ---
2
--- 0.6371021270751953 seconds ---
3
--- 0.8313305377960205 seconds ---
4
--- 0.5133256912231445 seconds ---
5
--- 0.4631667137145996 seconds ---
6
--- 0.4840662479400635 seconds ---
7
--- 0.5939476490020752 seconds ---
8
--- 0.46265530586242676 seconds ---
9
--- 0.6720304489135742 seconds ---
10
--- 1.419053554534912 seconds ---
11
--- 0.6252682209014893 seconds ---
12
--- 0.7270071506500244 seconds ---
13
--- 0.5191137790679932 seconds ---
14
--- 0.5044796466827393 seconds ---
15
--- 0.35965633392333984 seconds ---
16
--- 0.5728874206542969 seconds ---
17
--- 1.0004119873046875 seconds ---
18
--- 0.5662980079650879 seconds ---
19
--- 0.5147149562835693 seconds ---
20
--- 0.8104605674743652 seconds ---
21
--- 0.4808032512664795 seconds ---
22
--- 0.4702029228210449 seconds ---
23
--- 0.4790055751800537 seconds ---
24
--- 0.5537207126617432 seconds ---
25
--- 0.5168113708496094 seconds ---
26
--- 0.626235723495

--- 0.7377924919128418 seconds ---
214
--- 0.47013068199157715 seconds ---
215
--- 0.5112476348876953 seconds ---
216
--- 0.5874309539794922 seconds ---
217
--- 0.583521842956543 seconds ---
218
--- 1.0440914630889893 seconds ---
219
--- 0.5209343433380127 seconds ---
220
--- 0.44728755950927734 seconds ---
221
--- 0.409135103225708 seconds ---
222
--- 0.6858518123626709 seconds ---
223
--- 0.5307574272155762 seconds ---
224
--- 0.4812016487121582 seconds ---
225
--- 0.4515249729156494 seconds ---
226
--- 0.7606556415557861 seconds ---
227
--- 0.47798776626586914 seconds ---
228
--- 0.5037381649017334 seconds ---
229
--- 0.46548962593078613 seconds ---
230
--- 0.5654237270355225 seconds ---
231
--- 0.4321768283843994 seconds ---
232
--- 0.6113348007202148 seconds ---
233
--- 0.42445802688598633 seconds ---
234
--- 0.5066590309143066 seconds ---
235
--- 0.48772549629211426 seconds ---
236
--- 0.4893467426300049 seconds ---
237
--- 0.4278595447540283 seconds ---
238
--- 0.465188264846801

--- 0.5008277893066406 seconds ---
424
--- 0.6965982913970947 seconds ---
425
--- 0.4401819705963135 seconds ---
426
--- 0.5658471584320068 seconds ---
427
--- 0.6476788520812988 seconds ---
428
--- 0.4406006336212158 seconds ---
429
--- 0.664808988571167 seconds ---
430
--- 0.5941550731658936 seconds ---
431
--- 0.7028946876525879 seconds ---
432
--- 0.532191276550293 seconds ---
433
--- 0.6457104682922363 seconds ---
434
--- 0.5487751960754395 seconds ---
435
--- 0.4927330017089844 seconds ---
436
--- 0.5691728591918945 seconds ---
437
--- 0.9752547740936279 seconds ---
438
--- 0.8824529647827148 seconds ---
439
--- 0.4709458351135254 seconds ---
440
--- 0.6721181869506836 seconds ---
441
--- 1.2482109069824219 seconds ---
442
--- 0.5071170330047607 seconds ---
443
--- 0.5684928894042969 seconds ---
444
--- 0.6407654285430908 seconds ---
445
--- 0.8503038883209229 seconds ---
446
--- 0.6964976787567139 seconds ---
447
--- 0.7243530750274658 seconds ---
448
--- 1.3757624626159668 seco

--- 0.43729186058044434 seconds ---
635
--- 0.7145493030548096 seconds ---
636
--- 0.4644205570220947 seconds ---
637
--- 0.6643218994140625 seconds ---
638
--- 0.6881530284881592 seconds ---
639
--- 0.4245755672454834 seconds ---
640
--- 0.6796104907989502 seconds ---
641
--- 0.47523021697998047 seconds ---
642
--- 0.4920969009399414 seconds ---
643
--- 0.6806285381317139 seconds ---
644
--- 0.8994123935699463 seconds ---
645
--- 0.5080509185791016 seconds ---
646
--- 0.5177547931671143 seconds ---
647
--- 0.5711936950683594 seconds ---
648
--- 0.6856520175933838 seconds ---
649
--- 0.7464165687561035 seconds ---
650
--- 0.771594762802124 seconds ---
651
--- 0.49466848373413086 seconds ---
652
--- 0.5756130218505859 seconds ---
653
--- 0.4992983341217041 seconds ---
654
--- 0.5089116096496582 seconds ---
655
--- 0.7909407615661621 seconds ---
656
--- 0.5890240669250488 seconds ---
657
--- 0.5075123310089111 seconds ---
658
--- 0.9235713481903076 seconds ---
659
--- 0.5659575462341309 

--- 0.5344102382659912 seconds ---
846
--- 0.45252275466918945 seconds ---
847
--- 0.48667168617248535 seconds ---
848
--- 1.041731595993042 seconds ---
849
--- 0.47119593620300293 seconds ---
850
--- 0.9087851047515869 seconds ---
851
--- 0.49793434143066406 seconds ---
852
--- 0.529184103012085 seconds ---
853
--- 0.5328185558319092 seconds ---
854
--- 0.5162231922149658 seconds ---
855
--- 0.5046286582946777 seconds ---
856
--- 0.8851420879364014 seconds ---
857
--- 0.7520191669464111 seconds ---
858
--- 0.598179817199707 seconds ---
859
--- 0.5281922817230225 seconds ---
860
--- 0.4906003475189209 seconds ---
861
--- 0.5187034606933594 seconds ---
862
--- 0.5399510860443115 seconds ---
863
--- 0.943248987197876 seconds ---
864
--- 0.5062346458435059 seconds ---
865
--- 0.4456319808959961 seconds ---
866
--- 0.6568427085876465 seconds ---
867
--- 0.5979166030883789 seconds ---
868
--- 0.6803016662597656 seconds ---
869
--- 0.6297550201416016 seconds ---
870
--- 0.9744992256164551 se

--- 0.685605525970459 seconds ---
1055
--- 0.4713327884674072 seconds ---
1056
--- 0.8140137195587158 seconds ---
1057
--- 0.5544850826263428 seconds ---
1058
--- 0.5784275531768799 seconds ---
1059
--- 0.8148188591003418 seconds ---
1060
--- 0.5154016017913818 seconds ---
1061
--- 0.5709879398345947 seconds ---
1062
--- 0.70603346824646 seconds ---
1063
--- 0.4779031276702881 seconds ---
1064
--- 0.5270328521728516 seconds ---
1065
--- 0.6203203201293945 seconds ---
1066
--- 0.6212778091430664 seconds ---
1067
--- 0.9217219352722168 seconds ---
1068
--- 0.4790489673614502 seconds ---
1069
--- 0.525146484375 seconds ---
1070
--- 0.5010781288146973 seconds ---
1071
--- 0.5684962272644043 seconds ---
1072
--- 0.5880706310272217 seconds ---
1073
--- 0.43500828742980957 seconds ---
1074
--- 0.5314157009124756 seconds ---
1075
--- 0.44902706146240234 seconds ---
1076
--- 0.6060795783996582 seconds ---
1077
--- 0.605457067489624 seconds ---
1078
--- 1.0947282314300537 seconds ---
1079
--- 0.

--- 0.5399715900421143 seconds ---
1260
--- 0.5005688667297363 seconds ---
1261
--- 0.4604172706604004 seconds ---
1262
--- 0.4929618835449219 seconds ---
1263
--- 0.7546238899230957 seconds ---
1264
--- 0.5152020454406738 seconds ---
1265
--- 0.4645099639892578 seconds ---
1266
--- 0.44107818603515625 seconds ---
1267
--- 0.4327554702758789 seconds ---
1268
--- 0.5231502056121826 seconds ---
1269
--- 0.5335290431976318 seconds ---
1270
--- 1.0739378929138184 seconds ---
1271
--- 0.4729797840118408 seconds ---
1272
--- 0.8658838272094727 seconds ---
1273
--- 0.5556461811065674 seconds ---
1274
--- 0.8269805908203125 seconds ---
1275
--- 0.8511383533477783 seconds ---
1276
--- 0.49853086471557617 seconds ---
1277
--- 0.6321189403533936 seconds ---
1278
--- 0.48232555389404297 seconds ---
1279
--- 0.5286204814910889 seconds ---
1280
--- 0.5461444854736328 seconds ---
1281
--- 0.6971619129180908 seconds ---
1282
--- 0.7189242839813232 seconds ---
1283
--- 0.5122628211975098 seconds ---
12

In [106]:
print('--- occ before ---')
print('Chamfer Distacne: m_', np.mean(cd_gt_v_b_list), ' s_', np.std(cd_gt_v_b_list))
print()
print('Earth Movers Distance: m_', np.mean(emd_gt_v_b_list), ' s_', np.std(emd_gt_v_b_list))
print()
print('Intersection over Union: m_', np.mean(iou_gt_v_b_list), ' s_', np.std(iou_gt_v_b_list))
print()

--- occ before ---
Chamfer Distacne: m_ 0.019340027  s_ 0.008348005

Earth Movers Distance: m_ 0.028405800926115096  s_ 0.016063361493914588

Intersection over Union: m_ 0.6053374187627271  s_ 0.14214165649988983

